In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import re
import requests
import json# adding a column that joins all reviews together into one big string
from textblob import TextBlob

In [4]:
lyric_df = pd.read_excel('lyrics database.xlsx')
lyric_df

,Unnamed: 0,Year,Rank,Artist,Song,Lyrics
0,0,2000,1,Faith Hill,Breathe,"['feel', 'magic', 'floating', 'air', 'gets', '..."
1,1,2000,2,Santana,Smooth,"['man', 'hot', 'one', 'like', 'seven', 'inches..."
2,2,2000,3,Santana,Maria Maria,"['ladies', 'gents', 'turn', 'sound', 'systems'..."
3,3,2000,4,Joe,I Wanna Know,"['yeah', 'ohoh', 'yeah', 'alright', 'oh', 'oh'..."
4,4,2000,5,Vertical Horizon,Everything You Want,"['somewhere', 'theres', 'speaking', 'already',..."
...,...,...,...,...,...,...
2295,2295,2022,96,Ernest,Flower Shops,"['beautiful', 'day', 'shes', 'crying', 'night'..."
2296,2296,2022,97,Jnr Choi,To the Moon,"['sit', 'talking', 'moon', 'teh', 'ha', 'pull'..."
2297,2297,2022,98,Sam Smith,Unholy,"['mummy', 'dont', 'know', 'daddys', 'getting',..."
2298,2298,2022,99,Kane Brown,One Mississippi,"['long', 'youve', 'youve', 'gone', 'many', 'ti..."


In [5]:
# Access spotify api
client_id = '96dcd39191cf46ee81bb5d1092e6716b'
client_secret = '22a0463e07e2485c8fc0a692d3412183'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
# adding the track result info to the dataframe
lyric_df['results'] = lyric_df['Song'].apply(lambda x: sp.search(q=x, type='track'))

In [7]:
# creating a blank column to match songs to id's with a loop
lyric_df['song_id'] = None

In [8]:
# will strip punctuation of names of 
def _strip_punctuation(string):
    """
    strips the punctuation from a string, including new line spacing
    
    Args:
        string(str): a string
        
    Returns:
        string(str): a cleaned string
    """
    
    # create a list of punctuations to remove
    punc_list = ['\n','.','?','!',',',':',';','(',')',"'"]
    
    # for every punctuation in the punctuation list, string the string of that punctuation
    for punc in punc_list:
        string = string.strip(punc)
    
    # return the string
    return string

In [9]:
# looping through the dataset to match song result and artists with the correct track id
for idx in range(len(lyric_df)):
    results = lyric_df['results'][idx]
    target = lyric_df.loc[idx, 'Artist']
    # want to make sure capitalization and punctuation don't interrupt a match
    target = target.lower()
    target = _strip_punctuation(target)
    # finding first artist and track name match in the api search results
    for result in results['tracks']['items']:
        name = result['artists'][0]['name']
        name = name.lower()
        name = _strip_punctuation(name)
        # saving the id when we find a match and moving to next
        if target in name:
            song_id = result['id']
            lyric_df.loc[idx,'song_id'] = song_id
            break

In [10]:
# dropping a few songs without the matching api info
lyric_df = lyric_df.dropna()

In [11]:
# pulling the track info into a column
lyric_df['track_info'] = lyric_df['song_id'].apply(lambda x: sp.track(x) if pd.notnull(x) else x)

<ipython-input-11-ed70236f19e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['track_info'] = lyric_df['song_id'].apply(lambda x: sp.track(x) if pd.notnull(x) else x)


In [12]:
# pulling the popularity score out of the column
lyric_df['popularity_score'] = lyric_df['track_info'].apply(lambda x: x['popularity'] if pd.notnull(x) else x)

<ipython-input-12-212d409778cb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['popularity_score'] = lyric_df['track_info'].apply(lambda x: x['popularity'] if pd.notnull(x) else x)


In [13]:
# pulling the album cover link
lyric_df['image_url'] = lyric_df['track_info'].apply(lambda x: x['album']['images'][0]['url'] if pd.notnull(x) else x)

<ipython-input-13-c948abc0f2cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['image_url'] = lyric_df['track_info'].apply(lambda x: x['album']['images'][0]['url'] if pd.notnull(x) else x)


In [14]:
# pulling a preview portion of the song out
lyric_df['preview_url'] = lyric_df['track_info'].apply(lambda x: x['preview_url'] if pd.notnull(x) else x)

<ipython-input-14-8cc8fd6ebe4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['preview_url'] = lyric_df['track_info'].apply(lambda x: x['preview_url'] if pd.notnull(x) else x)


In [15]:
# pulling out album type information from track_info
lyric_df['album_type'] = lyric_df['track_info'].apply(lambda x: x['album']['album_type'] if pd.notnull(x) else x)

<ipython-input-15-9dde5ada1577>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['album_type'] = lyric_df['track_info'].apply(lambda x: x['album']['album_type'] if pd.notnull(x) else x)


In [16]:
def _remove_stop_words(word_list):
        """
        removes stop words from the word list
        
        Args:
            word_list(list): a list of words originating from a file
            
        Returns:
            filtered_words(list): a list of filtered words
        """
        # will replace next two lines of code with the NLTK stuff
        
        # open the stopwords text file 
        with open('stopwords.txt') as txt:
            # split it by the new line code
            stop_words = txt.read().split('\n')
        
        # initialize a filtered_words list
        filtered_words = []
        
        # for each word in the wordlist, if not in the stop words, we add to the filtered words list
        for word in word_list:
            if word not in stop_words:
                filtered_words.append(word)
            else:
                # continue if word is within the stop_words list
                continue
                
        # return the filtered words
        return filtered_words

In [17]:
def find_senti_score(lyric_list):
    """
    """
    # create a filtered list
    filtered_list = _remove_stop_words(lyric_list)

    # gather a song-specific sentiment score
    doc_string = ' '.join(filtered_list)
    avg_sentiment = TextBlob(doc_string).sentiment.polarity
    
    return avg_sentiment

In [18]:
# tricking the df column type into being a list
lyric_df['Lyrics'] = lyric_df['Lyrics'].apply(lambda x: list(str(x).split(',')))

<ipython-input-18-1e4aacbfb84c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyric_df['Lyrics'] = lyric_df['Lyrics'].apply(lambda x: list(str(x).split(',')))


In [19]:
# finding the sentiment score of each song based on lyrics
lyric_df['sentiment_score'] = lyric_df['Lyrics'].apply(lambda x: find_senti_score(x))

FileNotFoundError: [Errno 2] No such file or directory: 'stopwords.txt'

In [ ]:
# pulling artist info out of the 
lyric_df['artist_info'] = lyric_df['Artist'].apply(lambda x: sp.search(q=x, type='artist') if pd.notnull(x) else x)

In [56]:
lyric_df['artist_info']['artists']

KeyError: 'artists'

In [31]:
lyric_df['artist_info'][0]['artists']['items'][0]['genres']

['contemporary country', 'country', 'country dawn', 'country road', 'pop rock']

In [44]:
# there is a bug but i got it to work in previous export
# importing that csv
export_df = pd.read_csv('export_music.csv')

In [52]:
len(lyric_df['song_id'].unique())

1850

In [45]:
genre_map = dict(export_df[['song_id', 'genres']].values)

In [53]:
len(genre_map)

1844

In [50]:
lyric_df['genres'] = lyric_df['song_id'].apply(lambda x: genre_map[x] if pd.notnull(x) else x)

KeyError: '046MBhhgQJJghnbgZCkaAR'

In [40]:
lyric_df['genres'] = lyric_df['artist_items'].apply(lambda x: x[0]['genres'] if pd.notnull(x[0]) else x)

AttributeError: 'list' object has no attribute 'any'

In [ ]:
lyric_df

In [ ]:
target_list = ['Unnamed: 0', 'Year', 'Rank', 'Artist', 'Song',
       'Lyrics', 'song_id', 'track_info', 'popularity_score',
       'image_url', 'preview_url', 'album_type', 'sentiment_score', 'genres', ]

export_df = lyric_df[target_list]
export_df.to_csv('export_music.csv')